In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.width',1000)

In [3]:
%%time
data = pd.read_csv('20200221_data.csv')

Wall time: 7.88 s


--------------------------------------------

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS

--------------------------------------

In [5]:
data.head(5)

,CarID,MakeId,Title,Year,Make,Model,Trim,BodyStyle,Rental,Location,GasMileage_city,GasMileage_highway,Price,Mileage,Transmission,FuelType,ExteriorColor,InteriorColor,DoorsNum,MaximumSeating,Engine,Drivetrain,VIN,OptionCount,MajorOptions,Certified,Accident Check,OwnershipHistory,SellingDays,OriginPrice
0,100351603,m1,"2007 Chevrolet Tahoe - $11,999",2007,Chevrolet,Tahoe,NaN,suv,NaN,TX,NaN,NaN,11999.0,129280.0,Automatic,Gasoline,Gold,NaN,NaN,NaN,V8,NaN,1GNFC13J47R120389,0,NaN,NaN,0.0,2.0,1873,12999.0
1,101113873,m1,"2005 Chevrolet Express - $11,995",2005,Chevrolet,Express,NaN,van,NaN,NH,NaN,NaN,11995.0,118113.0,Automatic,NaN,White,NaN,NaN,NaN,NaN,NaN,1GBJG31U251136588,0,NaN,NaN,1.0,1.0,1863,NaN
2,102996725,m1,2010 Chevrolet Silverado 2500HD Work Truck Ext...,2010,Chevrolet,Silverado 2500HD,Work Truck Extended Cab LB 4WD,pickup_truck,NaN,KY,NaN,NaN,15900.0,144000.0,Automatic,Gasoline,White,Gray,4 doors,6 seats,V8,Four-Wheel Drive,1GC5KVBG7AZ218246,2,"Steel Wheels,Tow Package",NaN,0.0,2.0,1837,16900.0
3,103384798,m1,2014 Chevrolet Corvette Stingray 3LT Coupe RWD...,2014,Chevrolet,Corvette,Stingray 3LT Coupe RWD,coupe,NaN,MT,17 MPG City,29 MPG Highway,49995.0,2500.0,7-Speed Manual,Gasoline,Red,Black,2 doors,2 seats,V8,Rear-Wheel Drive,1G1YE2D73E5104797,5,"Bluetooth,Backup Camera,Aluminum Wheels,Naviga...",NaN,0.0,1.0,1832,62900.0
4,104800849,m1,"2002 Chevrolet Monte Carlo SS FWD - $3,440",2002,Chevrolet,Monte Carlo,SS FWD,coupe,NaN,IL,17 MPG City,27 MPG Highway,3440.0,174000.0,Automatic,Gasoline,Black,Black,2 doors,5 seats,V6,Front-Wheel Drive,2G1WX15K829249749,2,"Leather Seats,Aluminum Wheels",NaN,0.0,2.0,1809,4800.0


In [6]:
data.columns

Index(['CarID', 'MakeId', 'Title', 'Year', 'Make', 'Model', 'Trim', 'BodyStyle', 'Rental', 'Location', 'GasMileage_city', 'GasMileage_highway', 'Price', 'Mileage', 'Transmission', 'FuelType', 'ExteriorColor', 'InteriorColor', 'DoorsNum', 'MaximumSeating', 'Engine', 'Drivetrain', 'VIN', 'OptionCount', 'MajorOptions', 'Certified', 'Accident Check', 'OwnershipHistory', 'SellingDays', 'OriginPrice'], dtype='object')

In [7]:
train_data = data.iloc[:,[0,3,4,7,9,12,13,23,26,27,28]]

In [8]:
train_data.isnull().sum()

CarID               0
Year                0
Make                0
BodyStyle           0
Location            0
Price               0
Mileage             0
OptionCount         0
Accident Check      0
OwnershipHistory    0
SellingDays         0
dtype: int64

In [9]:
train_data.dtypes

CarID                 int64
Year                  int64
Make                 object
BodyStyle            object
Location             object
Price               float64
Mileage             float64
OptionCount           int64
Accident Check      float64
OwnershipHistory    float64
SellingDays           int64
dtype: object

In [10]:
locations = pd.get_dummies(train_data.iloc[:,4])

In [11]:
train_data = train_data.drop(columns=['Location'])

In [12]:
train_data = pd.concat([train_data, locations], axis=1)

In [13]:
train_data.head()

,CarID,Year,Make,BodyStyle,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,100351603,2007,Chevrolet,suv,11999.0,129280.0,0,0.0,2.0,1873,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,101113873,2005,Chevrolet,van,11995.0,118113.0,0,1.0,1.0,1863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,102996725,2010,Chevrolet,pickup_truck,15900.0,144000.0,2,0.0,2.0,1837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,103384798,2014,Chevrolet,coupe,49995.0,2500.0,5,0.0,1.0,1832,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,104800849,2002,Chevrolet,coupe,3440.0,174000.0,2,0.0,2.0,1809,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
train_data.shape

(2151338, 60)

In [15]:
train_data.isnull().sum()

CarID               0
Year                0
Make                0
BodyStyle           0
Price               0
Mileage             0
OptionCount         0
Accident Check      0
OwnershipHistory    0
SellingDays         0
AL                  0
AR                  0
AZ                  0
CA                  0
CO                  0
CT                  0
DC                  0
DE                  0
FL                  0
GA                  0
IA                  0
ID                  0
IL                  0
IN                  0
KS                  0
KY                  0
LA                  0
MA                  0
MD                  0
ME                  0
MI                  0
MN                  0
MO                  0
MS                  0
MT                  0
NC                  0
ND                  0
NE                  0
NH                  0
NJ                  0
NM                  0
NV                  0
NY                  0
OH                  0
OK                  0
ON        

In [16]:
makes = pd.get_dummies(train_data.iloc[:,2])

In [17]:
bodystyles = pd.get_dummies(train_data.iloc[:,3])

In [18]:
train_data = train_data.drop(columns=['Make', 'BodyStyle'])

In [19]:
train_data = pd.concat([train_data, makes, bodystyles], axis=1)

In [20]:
train_data

,CarID,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,Acura,Alfa Romeo,Aston Martin,Audi,BMW,Bentley,Buick,Cadillac,Chevrolet,Chrysler,Dodge,FIAT,Ferrari,Ford,GMC,Genesis,Honda,Hummer,Hyundai,INFINITI,Isuzu,Jaguar,Jeep,Kia,Lamborghini,Land Rover,Lexus,Lincoln,MINI,Maserati,Mazda,McLaren,Mercedes-Benz,Mercury,Mitsubishi,Nissan,Oldsmobile,Plymouth,Pontiac,Porsche,Ram,Rolls-Royce,SRT,Saab,Saturn,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo,smart,convertible,coupe,crossover,hatchback,minivan,pickup_truck,sedan,suv,van,wagon
0,100351603,2007,11999.0,129280.0,0,0.0,2.0,1873,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,101113873,2005,11995.0,118113.0,0,1.0,1.0,1863,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,102996725,2010,15900.0,144000.0,2,0.0,2.0,1837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,103384798,2014,49995.0,2500.0,5,0.0,1.0,1832,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,104800849,2002,3440.0,174000.0,2,0.0,2.0,1809,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151333,261405061,2016,13916.0,27479.0,4,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2151334,261406804,2013,5500.0,69896.0,0,0.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2151335,261410367,2016,9430.0,18301.0,4,0.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2151336,261410368,2016,10150.0,10006.0,3,0.0,1.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


-------------------------------------------------------------

In [27]:
part = train_data.sample(frac=0.01)

In [28]:
part.shape

(21513, 121)

In [29]:
data_for_train = part.drop(columns=['CarID'])

In [30]:
data_for_train.shape

(21513, 120)

------------------------

In [31]:
mms = MinMaxScaler()

In [32]:
data_for_train

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,Acura,Alfa Romeo,Aston Martin,Audi,BMW,Bentley,Buick,Cadillac,Chevrolet,Chrysler,Dodge,FIAT,Ferrari,Ford,GMC,Genesis,Honda,Hummer,Hyundai,INFINITI,Isuzu,Jaguar,Jeep,Kia,Lamborghini,Land Rover,Lexus,Lincoln,MINI,Maserati,Mazda,McLaren,Mercedes-Benz,Mercury,Mitsubishi,Nissan,Oldsmobile,Plymouth,Pontiac,Porsche,Ram,Rolls-Royce,SRT,Saab,Saturn,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo,smart,convertible,coupe,crossover,hatchback,minivan,pickup_truck,sedan,suv,van,wagon
1631346,2012,17500.0,110787.0,3,0.0,2.0,42,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1816944,2016,20895.0,31742.0,6,0.0,1.0,148,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1649657,2019,37174.0,5378.0,9,0.0,1.0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
742998,2016,16891.0,72450.0,4,0.0,3.0,19,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1458804,2008,2995.0,168000.0,1,0.0,5.0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045880,2019,44000.0,22113.0,10,0.0,1.0,78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
252671,2015,23800.0,34834.0,6,0.0,1.0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
21957,2004,6500.0,164398.0,0,2.0,1.0,183,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1187240,2017,14981.0,29835.0,4,0.0,1.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [33]:
data_for_train.iloc[:,[0,1,2,3,4,5,6]]

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays
1631346,2012,17500.0,110787.0,3,0.0,2.0,42
1816944,2016,20895.0,31742.0,6,0.0,1.0,148
1649657,2019,37174.0,5378.0,9,0.0,1.0,16
742998,2016,16891.0,72450.0,4,0.0,3.0,19
1458804,2008,2995.0,168000.0,1,0.0,5.0,6
...,...,...,...,...,...,...,...
1045880,2019,44000.0,22113.0,10,0.0,1.0,78
252671,2015,23800.0,34834.0,6,0.0,1.0,3
21957,2004,6500.0,164398.0,0,2.0,1.0,183
1187240,2017,14981.0,29835.0,4,0.0,1.0,5


In [34]:
%%time
data_for_train.iloc[:,[0,1,2,3,4,5,6]] = mms.fit_transform(data_for_train.iloc[:,[0,1,2,3,4,5,6]])

Wall time: 18 ms


In [35]:
data_for_train.iloc[:,[0,1,2,3,4,5,6]]

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays
1631346,0.794872,0.047955,0.284923,0.09375,0.0,0.076923,0.023217
1816944,0.897436,0.057504,0.081543,0.18750,0.0,0.000000,0.081813
1649657,0.974359,0.103291,0.013709,0.28125,0.0,0.000000,0.008845
742998,0.897436,0.046243,0.186283,0.12500,0.0,0.153846,0.010503
1458804,0.692308,0.007158,0.432130,0.03125,0.0,0.307692,0.003317
...,...,...,...,...,...,...,...
1045880,0.974359,0.122490,0.056767,0.31250,0.0,0.000000,0.043118
252671,0.871795,0.065675,0.089498,0.18750,0.0,0.000000,0.001658
21957,0.589744,0.017016,0.422862,0.00000,0.4,0.000000,0.101161
1187240,0.923077,0.040870,0.076636,0.12500,0.0,0.000000,0.002764


In [36]:
import math

In [37]:
1/math.sqrt(2)

0.7071067811865475

In [38]:
data_for_train.iloc[:,7:] = data_for_train.iloc[:,7:]*0.7

In [39]:
data_for_train = data_for_train.astype('float32')

In [40]:
data_for_train.shape

(21513, 120)

In [41]:
data_for_train.head(10)

,Year,Price,Mileage,OptionCount,Accident Check,OwnershipHistory,SellingDays,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,ON,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,Acura,Alfa Romeo,Aston Martin,Audi,BMW,Bentley,Buick,Cadillac,Chevrolet,Chrysler,Dodge,FIAT,Ferrari,Ford,GMC,Genesis,Honda,Hummer,Hyundai,INFINITI,Isuzu,Jaguar,Jeep,Kia,Lamborghini,Land Rover,Lexus,Lincoln,MINI,Maserati,Mazda,McLaren,Mercedes-Benz,Mercury,Mitsubishi,Nissan,Oldsmobile,Plymouth,Pontiac,Porsche,Ram,Rolls-Royce,SRT,Saab,Saturn,Scion,Subaru,Suzuki,Tesla,Toyota,Volkswagen,Volvo,smart,convertible,coupe,crossover,hatchback,minivan,pickup_truck,sedan,suv,van,wagon
1631346,0.794872,0.047955,0.284923,0.09375,0.0,0.076923,0.023217,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1816944,0.897436,0.057504,0.081543,0.18750,0.0,0.000000,0.081813,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0
1649657,0.974359,0.103291,0.013709,0.28125,0.0,0.000000,0.008845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
742998,0.897436,0.046243,0.186283,0.12500,0.0,0.153846,0.010503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458804,0.692308,0.007158,0.432130,0.03125,0.0,0.307692,0.003317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249894,0.794872,0.053297,0.258846,0.21875,0.0,0.230769,0.002211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0
1131238,0.923077,0.054720,0.054254,0.12500,0.0,0.000000,0.046987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,

-------------------

##### 模型

In [42]:
db1 = DBSCAN(eps=0.2,
    min_samples=5,
    metric='euclidean',
    n_jobs=6)

In [43]:
%%time
y_db1 = db1.fit_predict(data_for_train)

Wall time: 11.2 s


In [44]:
np.unique(y_db1)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 17

In [45]:
unique_db1, counts_db1 = np.unique(y_db1, return_counts=True)

In [46]:
dict(zip(unique_db1, counts_db1))

{-1: 11003,
 0: 6,
 1: 33,
 2: 16,
 3: 7,
 4: 10,
 5: 20,
 6: 18,
 7: 6,
 8: 35,
 9: 27,
 10: 11,
 11: 21,
 12: 44,
 13: 17,
 14: 15,
 15: 7,
 16: 29,
 17: 7,
 18: 11,
 19: 15,
 20: 11,
 21: 13,
 22: 107,
 23: 27,
 24: 7,
 25: 20,
 26: 85,
 27: 7,
 28: 13,
 29: 9,
 30: 67,
 31: 24,
 32: 10,
 33: 10,
 34: 9,
 35: 13,
 36: 14,
 37: 14,
 38: 14,
 39: 12,
 40: 6,
 41: 14,
 42: 30,
 43: 26,
 44: 22,
 45: 32,
 46: 8,
 47: 13,
 48: 30,
 49: 14,
 50: 16,
 51: 32,
 52: 15,
 53: 50,
 54: 28,
 55: 12,
 56: 21,
 57: 26,
 58: 5,
 59: 32,
 60: 15,
 61: 5,
 62: 16,
 63: 5,
 64: 8,
 65: 22,
 66: 7,
 67: 14,
 68: 9,
 69: 17,
 70: 5,
 71: 11,
 72: 12,
 73: 27,
 74: 17,
 75: 8,
 76: 49,
 77: 5,
 78: 18,
 79: 19,
 80: 6,
 81: 25,
 82: 10,
 83: 10,
 84: 8,
 85: 13,
 86: 16,
 87: 50,
 88: 25,
 89: 10,
 90: 46,
 91: 6,
 92: 23,
 93: 12,
 94: 29,
 95: 9,
 96: 47,
 97: 16,
 98: 7,
 99: 16,
 100: 16,
 101: 16,
 102: 11,
 103: 12,
 104: 5,
 105: 46,
 106: 6,
 107: 8,
 108: 32,
 109: 9,
 110: 8,
 111: 8,
 112: 9,

----------------------------------------

In [47]:
db2 = DBSCAN(eps=0.3,
    min_samples=5,
    metric='euclidean',
    n_jobs=6)

In [48]:
%%time
y_db2 = db2.fit_predict(data_for_train)

Wall time: 15.2 s


In [49]:
np.unique(y_db2)

array([  -1,    0,    1, ..., 1114, 1115, 1116], dtype=int64)

In [50]:
unique_db2, counts_db2 = np.unique(y_db2, return_counts=True)

In [51]:
dict(zip(unique_db2, counts_db2))

{-1: 7099,
 0: 12,
 1: 42,
 2: 56,
 3: 20,
 4: 11,
 5: 13,
 6: 26,
 7: 20,
 8: 7,
 9: 39,
 10: 10,
 11: 32,
 12: 12,
 13: 29,
 14: 56,
 15: 22,
 16: 19,
 17: 9,
 18: 32,
 19: 7,
 20: 15,
 21: 15,
 22: 15,
 23: 9,
 24: 27,
 25: 18,
 26: 15,
 27: 29,
 28: 131,
 29: 28,
 30: 8,
 31: 20,
 32: 101,
 33: 7,
 34: 15,
 35: 6,
 36: 9,
 37: 76,
 38: 31,
 39: 12,
 40: 13,
 41: 7,
 42: 18,
 43: 19,
 44: 26,
 45: 14,
 46: 19,
 47: 16,
 48: 17,
 49: 19,
 50: 17,
 51: 12,
 52: 18,
 53: 20,
 54: 40,
 55: 5,
 56: 31,
 57: 27,
 58: 38,
 59: 9,
 60: 17,
 61: 5,
 62: 14,
 63: 16,
 64: 9,
 65: 30,
 66: 19,
 67: 5,
 68: 6,
 69: 17,
 70: 6,
 71: 41,
 72: 22,
 73: 5,
 74: 55,
 75: 32,
 76: 18,
 77: 14,
 78: 23,
 79: 32,
 80: 6,
 81: 9,
 82: 5,
 83: 38,
 84: 6,
 85: 21,
 86: 7,
 87: 18,
 88: 5,
 89: 10,
 90: 31,
 91: 10,
 92: 15,
 93: 11,
 94: 21,
 95: 6,
 96: 6,
 97: 14,
 98: 16,
 99: 28,
 100: 17,
 101: 19,
 102: 13,
 103: 5,
 104: 7,
 105: 6,
 106: 21,
 107: 7,
 108: 33,
 109: 12,
 110: 14,
 111: 14,
 112: 

-------------------------------------------------------------

In [52]:
opt1 = OPTICS(min_samples=5,
    max_eps=0.2,
    metric='minkowski',
    p=2,
    cluster_method='dbscan',
    eps=0.2,
    n_jobs=6)

In [53]:
%%time
y_opt1 = opt1.fit_predict(data_for_train)

Wall time: 19min 9s


In [54]:
np.unique(y_opt1)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 17

In [55]:
unique_opt1, counts_opt1 = np.unique(y_opt1, return_counts=True)

In [56]:
dict(zip(unique_opt1, counts_opt1))

{-1: 11354,
 0: 6,
 1: 33,
 2: 16,
 3: 7,
 4: 10,
 5: 20,
 6: 18,
 7: 6,
 8: 35,
 9: 27,
 10: 11,
 11: 21,
 12: 44,
 13: 17,
 14: 15,
 15: 7,
 16: 29,
 17: 7,
 18: 11,
 19: 15,
 20: 11,
 21: 13,
 22: 107,
 23: 27,
 24: 7,
 25: 20,
 26: 85,
 27: 7,
 28: 13,
 29: 9,
 30: 67,
 31: 24,
 32: 10,
 33: 10,
 34: 9,
 35: 13,
 36: 14,
 37: 14,
 38: 14,
 39: 12,
 40: 6,
 41: 14,
 42: 30,
 43: 26,
 44: 22,
 45: 32,
 46: 8,
 47: 13,
 48: 30,
 49: 14,
 50: 16,
 51: 32,
 52: 15,
 53: 50,
 54: 28,
 55: 12,
 56: 21,
 57: 26,
 58: 5,
 59: 32,
 60: 15,
 61: 5,
 62: 16,
 63: 5,
 64: 8,
 65: 22,
 66: 7,
 67: 14,
 68: 9,
 69: 17,
 70: 5,
 71: 11,
 72: 12,
 73: 27,
 74: 17,
 75: 8,
 76: 48,
 77: 5,
 78: 18,
 79: 19,
 80: 6,
 81: 25,
 82: 10,
 83: 10,
 84: 8,
 85: 13,
 86: 16,
 87: 50,
 88: 25,
 89: 10,
 90: 46,
 91: 6,
 92: 23,
 93: 12,
 94: 29,
 95: 9,
 96: 47,
 97: 16,
 98: 7,
 99: 16,
 100: 16,
 101: 16,
 102: 11,
 103: 12,
 104: 5,
 105: 46,
 106: 6,
 107: 8,
 108: 32,
 109: 9,
 110: 8,
 111: 8,
 112: 9,

--------------------------------------------------------------------------------

In [57]:
opt2 = OPTICS(min_samples=50,
    max_eps=0.2,
    metric='minkowski',
    p=2,
    cluster_method='xi',
    eps=None,
    xi=0.05,
    n_jobs=6)

In [58]:
%%time
y_opt2 = opt2.fit_predict(data_for_train)

Wall time: 38.9 s


In [59]:
np.unique(y_opt2)

array([-1,  0,  1,  2,  3])

In [60]:
unique_opt2, counts_opt2 = np.unique(y_opt2, return_counts=True)

In [61]:
dict(zip(unique_opt2, counts_opt2))

{-1: 21247, 0: 74, 1: 60, 2: 77, 3: 55}

--------------------------------

In [62]:
opt3 = OPTICS(min_samples=100,
    max_eps=0.2,
    metric='minkowski',
    p=2,
    cluster_method='xi',
    eps=None,
    xi=0.05,
    n_jobs=6)

In [63]:
%%time
y_opt3 = opt3.fit_predict(data_for_train)

Wall time: 19.9 s


c:\program files\python37\lib\site-packages\sklearn\cluster\optics_.py:491: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  UserWarning)


In [64]:
np.unique(y_opt3)

array([-1])

In [65]:
unique_opt3, counts_opt3 = np.unique(y_opt3, return_counts=True)

In [66]:
dict(zip(unique_opt3, counts_opt3))

{-1: 21513}

------------------------------------------------

In [67]:
opt4 = OPTICS(min_samples=50,
    max_eps=0.3,
    metric='minkowski',
    p=2,
    cluster_method='xi',
    eps=None,
    xi=0.05,
    n_jobs=6)

In [68]:
%%time
y_opt4 = opt4.fit_predict(data_for_train)

Wall time: 1min 26s


In [69]:
np.unique(y_opt4)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [70]:
unique_opt4, counts_opt4 = np.unique(y_opt4, return_counts=True)

In [71]:
dict(zip(unique_opt4, counts_opt4))

{-1: 20696,
 0: 126,
 1: 97,
 2: 75,
 3: 51,
 4: 60,
 5: 55,
 6: 53,
 7: 61,
 8: 78,
 9: 60,
 10: 51,
 11: 50}

----------------------

In [77]:
opt5 = OPTICS(min_samples=50,
    max_eps=0.5,
    metric='minkowski',
    p=2,
    cluster_method='xi',
    eps=None,
    xi=0.05,
    n_jobs=6)

In [78]:
%%time
y_opt5 = opt5.fit_predict(data_for_train)

Wall time: 2min 48s


In [79]:
np.unique(y_opt5)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19])

In [80]:
unique_opt5, counts_opt5 = np.unique(y_opt5, return_counts=True)

In [81]:
dict(zip(unique_opt5, counts_opt5))

{-1: 20192,
 0: 56,
 1: 57,
 2: 134,
 3: 101,
 4: 77,
 5: 56,
 6: 62,
 7: 57,
 8: 58,
 9: 56,
 10: 57,
 11: 62,
 12: 52,
 13: 86,
 14: 56,
 15: 62,
 16: 61,
 17: 61,
 18: 56,
 19: 54}